In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

%cd "/content/drive/MyDrive/CONSEGNA_ML/"


In [ ]:
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as T
from torchvision import transforms
import torch
from torch import nn, optim
from models import DeepLabV3, CNN_7_Layers, DeepLabV3Lite
from tqdm import tqdm
from utils import ImageSegmentationDatasetOneHotEncoding, ImageSegmentationDatasetLogit, color_map
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
from PIL import Image
from torchvision.transforms.functional import to_pil_image
import matplotlib.pyplot as plt

In [ ]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

## Load dataset

In [ ]:
# Full dataset
# train_dir = '../../datasets/esame_ml/train'
train_dir = './train' # Colab
dataset = ImageSegmentationDatasetLogit(root_dir=train_dir)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

torchvision transforms are not designed to work on both image and label synchronously : we use a boolean to do it in the loading

In [ ]:
# Data split
val_percent = 0.2
val_size = int(len(dataset) * val_percent)
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

## Functions for training

In [ ]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0

    loop = tqdm(dataloader, desc="Training", leave=False)

    for rgb, _, labels in loop:
        rgb, labels = rgb.to(device), labels.to(device)

        #targets = labels.argmax(dim=1)  # (B, C, H, W) → (B, H, W) - to keep if we use one hot encoding
        targets=labels.long()

        optimizer.zero_grad()
        outputs = model(rgb)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    return running_loss / len(dataloader)

In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for rgb, _, labels in dataloader:
            rgb, labels = rgb.to(device), labels.to(device)
            #targets = labels.argmax(dim=1)
            targets=labels.long()
            outputs = model(rgb)
            loss = criterion(outputs, targets)

            total_loss += loss.item()

    return total_loss / len(dataloader)

shape [16, 544, 1024] vs [16, 9, 544, 1024] aka input [16, 9, 544, 1024] vs target [16, 544, 1024]

normal for CE : "Input (C), (C,N), (C,N,d_1,...,d_k)" "Target (), (N), (N,d_1,...,d_k)" in the documentation

CE needs class indices

## Loop

In [ ]:
x = torch.randn(1, 3, 224, 224).to('cuda')
print("Tensor on GPU:", x.device)

Tensor on GPU: cuda:0


In [ ]:
#!nvidia-smi

In [ ]:
for _,_, labels in train_loader:
    print(labels.shape, labels.min(), labels.max(), labels.dtype)
    break

torch.Size([4, 544, 1024]) tensor(0, dtype=torch.uint8) tensor(8, dtype=torch.uint8) torch.uint8


In [ ]:
def load_model_from_checkpoint():
    checkpoint = torch.load('checkpoint_epoch_10.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # start_epoch = checkpoint['epoch'] + 1
    return model, start_epoch

In [ ]:
# Hyperparameters
EPOCHS = 15
LR = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

start_epoch = 0
model = DeepLabV3Lite().to(device)
# model, start_epoch = load_model_from_checkpoint()

optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

# Weights for the loss - weighted loss
class_weights = torch.tensor([
  class_weights[0] = 1.5      # undefined/background - white
  class_weights[1] = 1.0      # smooth trail - grey
  class_weights[2] = 2.0      # traversable grass - light green
  class_weights[3] = 1.0      # rough trail - brown
  class_weights[4] = 1.0      # puddle - pink
  class_weights[5] = 1.0      # obstacle - red
  class_weights[6] = 2.0      # non-traversable low vegetation - medium green
  class_weights[7] = 1.0      # high vegetation - dark green
  class_weights[8] = 1.0      # sky - blue
], dtype=torch.float).to(device)

# Loss
criterion = nn.CrossEntropyLoss(weight=class_weights)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Loop
for epoch in range(start_epoch, EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    scheduler.step(val_loss)
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    # Save every 5 epochs
    if (epoch + 1) % 5 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, f'checkpoint_epoch_{epoch+1}.pth')

Epoch 2/50 | Train Loss: 0.9500 | Val Loss: 1.3592


Epoch 3/50 | Train Loss: 0.7451 | Val Loss: 0.7809


Epoch 4/50 | Train Loss: 0.6777 | Val Loss: 0.7496


Epoch 5/50 | Train Loss: 0.6334 | Val Loss: 0.6599


Epoch 6/50 | Train Loss: 0.6176 | Val Loss: 0.6695


...

Epoch 46/50 | Train Loss: 0.1309 | Val Loss: 0.8601


Epoch 47/50 | Train Loss: 0.1282 | Val Loss: 0.8422


Epoch 48/50 | Train Loss: 0.1295 | Val Loss: 0.8757


Epoch 49/50 | Train Loss: 0.1300 | Val Loss: 0.8611


Epoch 50/50 | Train Loss: 0.1271 | Val Loss: 0.8953


## Validation visualization

In [ ]:
def convert_prediction_to_rgb(pred, color_map):
    class_map = pred.argmax(dim=0).cpu().numpy()  # (H, W)
    inverse_color_map = {v: k for k, v in color_map.items()}
    h, w = class_map.shape
    rgb = np.zeros((h, w, 3), dtype=np.uint8)
    for class_idx, color in inverse_color_map.items():
        rgb[class_map == class_idx] = color
    return rgb

In [ ]:
def visualize_sample_prediction(model, rgb_tensor, label_tensor, color_map, device):
    image_tensor = rgb_tensor.unsqueeze(0).to(device)  # (1, 3, H, W)
    rgb_image = image_tensor.squeeze()

    model = model.to(device)
    model.eval()

    with torch.no_grad():
        output = model(image_tensor)  # (1, 9, H, W)
        output = output.squeeze(0).softmax(dim=0)  # (9, H, W)

    pred_rgb = convert_prediction_to_rgb(output, color_map)

    rgb_image = to_pil_image(rgb_image.cpu())
    label_image = to_pil_image(label_tensor.cpu())

    # Plot all
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("Input Image")
    plt.imshow(rgb_image)
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title("Ground Truth")
    plt.imshow(label_image)
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title("Model Prediction")
    plt.imshow(pred_rgb)
    plt.axis("off")

    plt.show()

In [ ]:
# if we want to check previous checkpoints with the same validation set

checkpoint_vis = torch.load('checkpoint_epoch_15.pth')
model_vis = DeepLabV3Lite().to(device)
model_vis.load_state_dict(checkpoint_vis['model_state_dict'])
model_vis.eval();

In [ ]:
for rgb_batch, label_batch, _ in val_loader:
    batch_size = rgb_batch.size(0)
    for i in range(min(5, batch_size)):
        rgb_tensor = rgb_batch[i]
        label_tensor = label_batch[i]
        visualize_sample_prediction(model_vis, rgb_tensor, label_tensor, color_map, device)
    #break